# Data manipulation in Pandas

In this notebook we'll introduce some of the basic concepts of the Pandas library for data manipulation. We'll also see simple chart examples using both Pandas and Seaborn.

Pandas holds data in memory. As a general rule of thumb, you should have 5-10 times as much RAM as the size of your dataset.

Be aware that some operations in Pandas are computationally expensive - we won't cover the details here, but you can sometimes speed up your code by orders of magnitude just by doing things in a slightly different way...

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
pd.__version__

Lots of things are changing in pandas... Version 1.0 is due for release in mid-2019, and will see some big changes. See [this article](https://www.dataschool.io/future-of-pandas/) for an overview. In particular, `inplace` is being deprecated, and the back end will eventually change to Apache Arrow, to improve performance, and because:

    "The more you know about the internals of pandas DataFrame,
    the more horrified you are."

    Wes McKinney @ SciPy 2018

## Basic Pandas data structures

In [ ]:
# The most important Pandas objects are the `Series` and the `DataFrame`
s = pd.Series({'x': 10, 'y': 20, 'z': 30})
s

In [ ]:
# We can slice a series in a similar way to a Python list
s[:2]

In [ ]:
# We can also select using the index
s['y']

In [ ]:
s[['x', 'z']]

When slicing a series, pandas returns a copy, similar to what we saw with python lists

In [ ]:
s = pd.Series(np.random.randn(100))
s.head()

In [ ]:
s2 = s[:10]
s2.head()

In [ ]:
s2[:] = 99

In [ ]:
s2.head()

In [ ]:
s.head()

But this behaviour becomes more complex when we use other slicing / indexing methods, or for DataFrames. If in doubt, do it explicitly instead of relying on implicit behaviour.

### DataFrames

A DataFrame is like a SAS dataset or a R dataframe (or tibble). Each column of a DataFrame is a Series.

In [ ]:
df = pd.DataFrame({'x': np.random.randn(100), 
                   'y': np.random.randint(0, 100, 100),
                   'z': np.random.choice(list('abcde'), 100)})

In [ ]:
df.head()

In [ ]:
df.dtypes

### Indexing and selection

We are only going to scratch the surface of selecting from Series and DataFrames. For more info, see the Pandas documentation or (better) the [Python Data Science Handbook, chapter 3](https://jakevdp.github.io/PythonDataScienceHandbook/03.00-introduction-to-pandas.html)

In [ ]:
# Lets set an index on the DataFrame:
df.set_index('z', inplace=True)
df.head()

In [ ]:
# Now we can select all records where the index = 'c':
df.loc['c'].head()

In [ ]:
# .loc looks at the "explicit" index - i.e. the one we've defined. In contrast, .iloc looks at the "implicit" 
# index, which is just the row number. So to select rows 5 to 9:
df.iloc[5:10]

You may see some examples on the web that use the .ix method for indexing. Don't do this - it's deprecated.

It's often unclear whether pandas will return a view or a copy of a series or dataframe. If in doubt, do it explicitly! For example, lets say we want to assign a new value to the 'x' column, for all rows where the index = 'c'. We might think we could do this by taking a slice and then assigning to it:

In [ ]:
df_slice = df.loc['c']
df_slice.head()

In [ ]:
# We could try to do this by assigning to the slice:
df_slice.loc['x'] = 99

If you see that warning message, you should probably look for another way of doing things. In this case, our slice was actually a copy, so pandas is warning us that we might not be doing what we think we're doing. Here's a better way:

In [ ]:
df.loc['c', 'x'] = 99

In [ ]:
df.loc['c'].head()

We'll see some more indexing examples below

### Read our dataset - the Tableau superstore data!

In [ ]:
try:
    superstore = pd.read_excel('data/superstore.xslx')
except FileNotFoundError:
    superstore = pd.read_excel('https://query.data.world/s/n2pyux2nabxy4c43zl3uugxsk5gt6v')

In [ ]:
# Quick check: do we have the right number of rows?
assert len(superstore) == 51290

In [ ]:
type(superstore)

In [ ]:
superstore.head()

### A few basic data exploration tasks

In [ ]:
# Basic summary of the table
superstore.info()

In [ ]:
# How many countries do we have?
superstore['Country'].nunique()

In [ ]:
# List of countries
superstore['Country'].value_counts()

In [ ]:
# Total sales
superstore['Sales'].sum()

In [ ]:
# Number of unique values for each column
superstore.nunique()

### Quick look at the distributions of numeric variables

In [ ]:
superstore.describe()

In [ ]:
plotdata = superstore.select_dtypes('number').drop('Postal Code', axis=1)
plotdata.dtypes

In [ ]:
g = sns.FacetGrid(plotdata.melt(), col='variable', col_wrap=3, sharey=False, sharex=False)
g.map(sns.distplot, 'value', kde=False)
plt.show()

### Aggregations: [Split, apply, combine](https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html#GroupBy:-Split,-Apply,-Combine)

In [ ]:
# Sales by segment
# select sum(Sales) from superstore group by Segment
superstore.groupby('Segment')['Sales'].sum()

In [ ]:
# Sales by market and segment
# select sum(Sales) from superstore group by Market, Segment
superstore.groupby(['Market', 'Segment'])['Sales'].sum()

In [ ]:
# We can store the results of a query in an object:
sales_summary = superstore.groupby(['Market', 'Segment'])['Sales'].sum()
type(sales_summary)

In [ ]:
# We now have an example of a 'multi-index' (or hierarchical index)
sales_summary.index

In [ ]:
# What were the sales figures for Asia Pacific?
sales_summary['Asia Pacific']

In [ ]:
# What were the sales for Consumer and Corporate segments in Europe?
sales_summary.loc[('Europe', ['Consumer', 'Corporate'])]

### Basic plotting: Sales by month

In [ ]:
superstore.set_index('Order Date').resample('1M')['Sales'].sum().plot();

### A bar chart: Sales by market

This demonstrates how getting a chart to look just the way you want it can get very fiddly very quickly! That's where recent plotting libraries such as Seaborn, Plotly, Chartify etc can be better than the built-in Pandas plotting methods, or doing it from scratch in matplotlib. 

In [ ]:
# First, choose our bar colour. By default, pandas uses a different colour for each bar - nasty!
colours = sns.color_palette('tab20') 
bar_colour = colours[0]

In [ ]:
with sns.axes_style('darkgrid'):
    ax = superstore.groupby('Market')['Sales'].sum().sort_values().plot.barh(color=bar_colour)

    plt.title("Total sales by market (£000)")
    ax.yaxis.label.set_visible(False)
    
    # Now format the axis labels. It really shouldn't be this hard...
    vals = ax.get_xticks()
    ax.set_xticklabels(['{:,.0f}'.format(x/1000) for x in vals]) 

### Transformations: creating new columns etc

In [ ]:
# Group by ... and create profit ratio column (profit / sales)
grouped = superstore.groupby('Segment')[['Sales', 'Profit']].sum()
grouped['profit_ratio'] = grouped['Profit'] / grouped['Sales']
grouped

Merges etc. Contrast SQL - which has to run on a database - with Pandas, which runs in memory. 
